In [11]:
data Exp = Val Int     -- n
         | Add Exp Exp -- e1 + e2
         | Sub Exp Exp -- e1 - e2
                       -- if ..
         deriving Show

In [12]:
eval :: Exp -> Int
eval (Val n) = n
eval (Add e1 e2) = eval e1 + eval e2
eval (Sub e1 e2) = eval e1 - eval e2

In [13]:
-- 3 + (10 - (2 + 5))
e = Val 3 `Add` (Val 10 `Sub` Add (Val 2) (Val 5))

In [14]:
eval e

6

인터프리터 실행중에 몇번 덧셈을 수행하나?

In [15]:
--     (입력식, 상태) ->   (결과값, 상태)
eval' :: (Exp, Int) -> (Int, Int)
eval' (Val n,     i) = (n, i)
eval' (Add e1 e2, i) = (v1+v2, k+1)
  where
  (v1, j) = eval' (e1, i)
  (v2, k) = eval' (e2, j)
eval' (Sub e1 e2, i) = (v1-v2, k)
  where
  (v1, j) = eval' (e1, i)
  (v2, k) = eval' (e2, j)

In [16]:
e

Add (Val 3) (Sub (Val 10) (Add (Val 2) (Val 5)))

In [17]:
eval' (e,0)

(6,2)

In [18]:
:info Monad

class Applicative m => Monad (m :: * -> *) where
  (>>=) :: m a -> (a -> m b) -> m b
  (>>) :: m a -> m b -> m b
  return :: a -> m a
  fail :: String -> m a
  {-# MINIMAL (>>=) #-}
  	-- Defined in ‘GHC.Base’
instance Monad (Either e) -- Defined in ‘Data.Either’
instance Monad IHaskellTH.Q -- Defined in ‘Language.Haskell.TH.Syntax’
instance Monad [] -- Defined in ‘GHC.Base’
instance Monad Maybe -- Defined in ‘GHC.Base’
instance Monad IO -- Defined in ‘GHC.Base’
instance Monad ((->) r) -- Defined in ‘GHC.Base’
instance Monoid a => Monad ((,) a) -- Defined in ‘GHC.Base’

In [39]:
{-# LANGUAGE DeriveFunctor #-}
-- 상태 ->   (결과값, 상태)
data IntState a = St (Int -> (a,Int)) deriving (Functor)

instance Applicative IntState where
  St h <*> St g = St (\i -> let (f,j) = h i -- 상태안의 함수 f를 꺼냄
                                (v,k) = g j -- 상태안의 값 v를 꺼냄
                             in (f v, k) )  -- 상태는 그냥 최종 상태로

instance Monad IntState where
  return v = St (\i -> (v,i))
  St g >>= f = St (\i -> let (v,j) = g i
                             St h  = f v
                          in h j)

runSt :: IntState a -> Int -> (a,Int)
runSt (St g) = g

-- 상태변환 고차함수
modifySt :: (Int -> Int) -> IntState ()
modifySt f = St (\i -> ((), f i))

inc = modifySt (+1)
{-
-- +1을 하는 상태 연산
inc :: IntState ()
inc = St (\i -> ((), i+1))
-}

In [27]:
ev :: Exp -> IntState Int
ev (Val n) = return n
ev (Add e1 e2) = do  v1 <- ev e1
                     v2 <- ev e2
                     inc -- ++
                     return (v1 + v2)
ev (Sub e1 e2) = (-) <$> ev e1 <*> ev e2

{-
  ev e1 >>= \v1 ->
  ev e2 >>= \v2 ->
  inc   >>= \_  ->
  return (v1 + v2) -}

In [28]:
e

Add (Val 3) (Sub (Val 10) (Add (Val 2) (Val 5)))

In [30]:
runSt (ev (Add (Val 3) (Val 2))) 0

runSt (ev (Add (Val 3) (Add (Val 10) (Add (Val 2) (Val 5))))) 0

runSt (ev e) 0

(5,1)

(20,3)

(6,2)

In [41]:
import Control.Monad.State

increase = modify (+1)

ev' (Val n) = return n
ev' (Add e1 e2) = do v1 <- ev' e1
                     v2 <- ev' e2
                     increase -- ++
                     return (v1 + v2)
ev' (Sub e1 e2) = (-) <$> ev' e1 <*> ev' e2

In [42]:
runState (ev' (Add (Val 3) (Val 2))) 0

runState (ev' (Add (Val 3) (Add (Val 10) (Add (Val 2) (Val 5))))) 0

runState (ev' e) 0

(5,1)

(20,3)

(6,2)

In [38]:
:info modify

modify :: MonadState s m => (s -> s) -> m () 	-- Defined in ‘Control.Monad.State.Class’